In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d omkargurav/face-mask-dataset

 97% 159M/163M [00:03<00:00, 70.1MB/s]
100% 163M/163M [00:03<00:00, 53.0MB/s]


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras import Sequential,layers


In [4]:
import zipfile
zip_ref=zipfile.ZipFile('/content/face-mask-dataset.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
import os

parent_dir='/content/data'
dir1='train'
dir2='test'
path1=os.path.join(parent_dir,dir1)
path2=os.path.join(parent_dir,dir2)

os.mkdir(path1)
os.mkdir(path2)


In [6]:
parent_dir='/content/data/train'
dir1='with_mask'
dir2='without_mask'
path1=os.path.join(parent_dir,dir1)
path2=os.path.join(parent_dir,dir2)

os.mkdir(path1)
os.mkdir(path2)

In [7]:
parent_dir='/content/data/test'
dir1='with_mask'
dir2='without_mask'
path1=os.path.join(parent_dir,dir1)
path2=os.path.join(parent_dir,dir2)

os.mkdir(path1)
os.mkdir(path2)

In [8]:
import os
import random
import shutil

source = '/content/data/with_mask'
dest = '/content/data/train/with_mask'
files = os.listdir(source)

no_of_files = len(files)-(len(files) // 30)

for file_name in random.sample(files, no_of_files):
    shutil.move(os.path.join(source, file_name), dest)

In [9]:
source = '/content/data/without_mask'
dest = '/content/data/train/without_mask'
files = os.listdir(source)
no_of_files = len(files)-(len(files) // 2)

for file_name in random.sample(files, no_of_files):
    shutil.move(os.path.join(source, file_name), dest)

In [10]:
source = '/content/data/without_mask'
dest = '/content/data/test/without_mask'
files = os.listdir(source)
no_of_files = len(files) // 30

for file_name in random.sample(files, no_of_files):
    shutil.move(os.path.join(source, file_name), dest)

In [11]:
source = '/content/data/with_mask'
dest = '/content/data/test/with_mask'
files = os.listdir(source)
no_of_files = len(files) // 30

for file_name in random.sample(files, no_of_files):
    shutil.move(os.path.join(source, file_name), dest)

In [12]:
import tensorflow as tf
train_df=tf.keras.utils.image_dataset_from_directory(
    directory='/content/data/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)

Found 5515 files belonging to 2 classes.


In [13]:
test_df=tf.keras.utils.image_dataset_from_directory(
    directory='/content/data/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)

Found 67 files belonging to 2 classes.


In [14]:
def process(image,labels):
  image=tf.cast(image/255,tf.float32)
  return image,labels

In [15]:
train_df=train_df.map(process)
test_df=test_df.map(process)

In [16]:
model=Sequential()

In [17]:
model.add(layers.Conv2D(32,(3,3),activation='relu',padding='valid',input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2,2),strides=2,padding='valid'))
model.add(layers.Conv2D(64,(3,3),activation='relu',padding='valid'))
model.add(layers.MaxPooling2D((2,2),strides=2,padding='valid'))
model.add(layers.Conv2D(128,(3,3),activation='relu',padding='valid'))


model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [19]:
model.compile(optimizer='adam',loss='BinaryCrossentropy',metrics=['accuracy'])
history=model.fit(train_df,epochs=5,validation_data=train_df)

Epoch 1/5
173/173 [==============================] - 27s 146ms/step - loss: 0.2337 - accuracy: 0.9041 - val_loss: 0.1792 - val_accuracy: 0.9229
Epoch 2/5
173/173 [==============================] - 25s 144ms/step - loss: 0.1809 - accuracy: 0.9278 - val_loss: 0.1378 - val_accuracy: 0.9507
Epoch 3/5
173/173 [==============================] - 26s 151ms/step - loss: 0.1291 - accuracy: 0.9494 - val_loss: 0.0819 - val_accuracy: 0.9688
Epoch 4/5
173/173 [==============================] - 25s 144ms/step - loss: 0.1045 - accuracy: 0.9637 - val_loss: 0.0743 - val_accuracy: 0.9686
Epoch 5/5
173/173 [==============================] - 25s 144ms/step - loss: 0.0748 - accuracy: 0.9723 - val_loss: 0.0521 - val_accuracy: 0.9801
